- **Desviación estándar muestral (no poblacional):**  
  Usar la distribución **t de Student**.

  $\bar{x} \pm t \cdot \frac{s}{\sqrt{n}}$

- **Desviación estándar poblacional:**  
  Usar la **normal** (`norm.ppf`) cuando el tamaño de la muestra es grande (**n > 30**) y la **t de Student** (`t.ppf`) cuando es pequeña (**n ≤ 30**).

  $\bar{x} \pm t \cdot \frac{s}{\sqrt{n}}$

  **a) Para la media:**  
  Se usa la **media muestral**, la **desviación estándar** (poblacional o muestral), el **tamaño de la muestra** y el **valor z** (o **t**).

- **Para proporciones muestrales:**  
  Usar la **fórmula específica para proporciones**, no la de medias.

  $\hat{p} \pm z \cdot \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$

  **b) Para proporciones:**  
  Se usa la **proporción muestral**, el **tamaño de la muestra** y el **valor z**.

### Desviación estándar muestral (no poblacional), debes usar la distribución t de Student, no la normal (norm.ppf)

In [ ]:
from scipy.stats import t as t_dist
t = t_dist.ppf(1 - alpha/2, df=n-1)  # valor crítico t
error = t * s / np.sqrt(n)    # error máximo donde s es la desviación estándar muestral y t es el valor crítico t
lim_inf = media_muestral - error
lim_sup = media_muestral + error
lim_inf, lim_sup, error

### Desviación estándar pooblacional: usar la normal (norm.ppf) cuando el tamaño de la muestra es grande (n > 30) y la t de Student (t.ppf) cuando es pequeña (n ≤ 30).

In [ ]:
# Datos del problema
n = 16                     # tamaño de la muestra
media_muestral = 27.5      # promedio de vida de la muestra
s = 5                      # desviación estándar muestral
confianza = 0.95           # nivel de confianza (95%)
alpha = 1 - confianza
gl = n - 1                 # grados de libertad

t = norm.ppf(1 - alpha/2)  # valor crítico t (aproximado con z para n>30)
error = t * s / np.sqrt(n) # error máximo donde s es la desviación estándar muestral y t es el valor crítico t
lim_inf = media_muestral - error
lim_sup = media_muestral + error
lim_inf, lim_sup, error

### Intervalos de Confianza para Proporciones

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, beta
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import samplesize_confint_proportion

plt.style.use('seaborn-v0_8-whitegrid')

def intervalo_proporcion_normal(x: int, n: int, confidence: float) -> dict:
    """Calcula intervalo de confianza normal aproximado para una proporción."""
    p_hat = x / n                         # estimador puntual de la proporción donde x es el número de éxitos y n el tamaño de la muestra
    alpha = 1 - confidence                # nivel de significancia
    z = norm.ppf(1 - alpha / 2)           # valor crítico z
    se = np.sqrt(p_hat * (1 - p_hat) / n) # error estándar
    error = z * se                        # error máximo
    return {
        "p_hat": p_hat,                   # estimador puntual
        "z": z,                           # valor crítico z
        "se": se,                         # error estándar
        "lower": max(0.0, p_hat - error), # límite inferior
        "upper": min(1.0, p_hat + error), # límite superior
        "error": error,                   # error máximo
        "confidence": confidence,         # nivel de confianza
        "n": n,                           # tamaño de la muestra
        "x": x                            # número de éxitos
    }

def intervalo_proporcion_clopper_pearson(x: int, n: int, confidence: float) -> tuple[float, float]:
    """Intervalo exacto (Clopper-Pearson) útil como referencia para muestras pequeñas."""
    alpha = 1 - confidence                                          # nivel de significancia
    lower = beta.ppf(alpha / 2, x, n - x + 1) if x > 0 else 0.0     # límite inferior donde x es el número de éxitos y n el tamaño de la muestra
    upper = beta.ppf(1 - alpha / 2, x + 1, n - x) if x < n else 1.0 # límite superior
    return lower, upper

def resumen_intervalo(nombre: str, resultado: dict, intervalo_exacta: tuple[float, float] | None = None) -> pd.DataFrame:
    """Arma un resumen tabular para reportar un intervalo de confianza."""
    data = {
        "escenario": [nombre],                                    # nombre del escenario
        "muestra (x/n)": [f"{resultado['x']}/{resultado['n']}"],  # tamaño de la muestra
        "p_con_hat": [resultado["p_hat"]],                        # estimador puntual
        "nivel_confianza": [resultado["confidence"]],             # nivel de confianza
        "limite_inferior": [resultado["lower"]],                  # límite inferior
        "limite_superior": [resultado["upper"]],                  # límite superior
        "error_maximo": [resultado["error"]]                      # error máximo
    }
    df = pd.DataFrame(data)
    if intervalo_exacta is not None:
        df["cp_inferior"], df["cp_superior"] = intervalo_exacta
    return df

Un fabricante produce focos que tienen un promedio de vida de distribución aproximadamente
normal y una desviacion estandar de 40 horas. Si una muestra de 30 focos tiene una vida
promedio de 780 horas. Encuentre el intervalo de confianza del 95%, 96%, 98% para la media de
la población de todos los focos que produce la empresa.

In [ ]:
'''Un fabricante produce focos que tienen un promedio de vida de distribución aproximadamente
normal y una desviacion estandar de 40 horas. Si una muestra de 30 focos tiene una vida
promedio de 780 horas. Encuentre el intervalo de confianza del 95%, 96%, 98% para la media de
la población de todos los focos que produce la empresa.'''

# Datos del problema
n = 30                # tamaño de la muestra
media_muestral = 780  # promedio de vida de la muestra
sigma = 40            # desviación estándar poblacional (conocida)
niveles_confianza = [0.95, 0.96, 0.98]

resultados = []
for conf in niveles_confianza:
    alpha = 1 - conf
    z = norm.ppf(1 - alpha/2)
    error = z * sigma / np.sqrt(n)
    lim_inf = media_muestral - error
    lim_sup = media_muestral + error
    resultados.append({
        "confianza": f"{int(conf*100)}%",
        "z": round(z, 3),
        "lim_inf": round(lim_inf, 2),
        "lim_sup": round(lim_sup, 2),
        "error_max": round(error, 2)
    })

df_resultados = pd.DataFrame(resultados)
df_resultados

Una compañía utiliza baterías en sus juegos electrónicos que según ellos duran un promedio de
30 horas, para confirmar esto, se prueba 16 baterías siendo la media muestral de 27.5 horas y
su desviación estándar S=5 horas. Encuentre un intervalo de confianza del 95% para la media.
Suponga que la distribución de las baterías es aproximadamente normal.

In [ ]:
'''Una compañía utiliza baterías en sus juegos electrónicos que según ellos duran un promedio de
30 horas, para confirmar esto, se prueba 16 baterías siendo la media muestral de 27.5 horas y
su desviación estándar S=5 horas. Encuentre un intervalo de confianza del 95% para la media.
Suponga que la distribución de las baterías es aproximadamente normal.'''

# Datos del problema
n = 16                     # tamaño de la muestra
media_muestral = 27.5      # promedio de vida de la muestra
s = 5                      # desviación estándar muestral
confianza = 0.95           # nivel de confianza (95%)
alpha = 1 - confianza
gl = n - 1                 # grados de libertad

t = norm.ppf(1 - alpha/2)  # valor crítico t (aproximado con z para n>30)
error = t * s / np.sqrt(n) # error máximo donde s es la desviación estándar muestral y t es el valor crítico t
lim_inf = media_muestral - error
lim_sup = media_muestral + error
lim_inf, lim_sup, error

#  distribución t de Student, para muestras pequeñas (n ≤ 30)
'''Para muestras pequeñas (n < 30) y desviación estándar muestral (no poblacional), debes usar la distribución t de Student, no la normal (norm.ppf)'''
from scipy.stats import t as t_dist
t = t_dist.ppf(1 - alpha/2, df=n-1)  # valor crítico t
error = t * s / np.sqrt(n)    # error máximo donde s es la desviación estándar muestral y t es el valor crítico t
lim_inf = media_muestral - error
lim_sup = media_muestral + error
lim_inf, lim_sup, error

En una muestra aleatoria de 20 porciones de cereal el contenido Promedio de azúcar fue de 11.3
gramos con desviación estándar de 2.45 gramos. Suponiendo que los contenidos de azúcar están
distribuidos normalmente, determine el intervalo de confianza del 95% para el contenido
promedio de azúcar en las porciones de dicho cereal.

In [ ]:
'''En una muestra aleatoria de 20 porciones de cereal el contenido Promedio de azúcar fue de 11.3
gramos con desviación estándar de 2.45 gramos. Suponiendo que los contenidos de azúcar están
distribuidos normalmente, determine el intervalo de confianza del 95% para el contenido
promedio de azúcar en las porciones de dicho cereal.'''

# Datos del problema
n = 20                     # tamaño de la muestra
media_muestral = 11.3      # promedio de contenido de azúcar de la muestra
s = 2.45                   # desviación estándar muestral
confianza = 0.95           # nivel de confianza (95%)
alpha = 1 - confianza
gl = n - 1                 # grados de libertad

t = t_dist.ppf(1 - alpha/2, df=n-1)  # valor crítico t
error = t * s / np.sqrt(n)            # error máximo donde s es la desviación estándar muestral y t es el valor crítico t
lim_inf = media_muestral - error
lim_sup = media_muestral + error
lim_inf, lim_sup, error